In [20]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
from sklearn.utils import shuffle


In [ ]:
def cargar_y_aplanar(folder_path, img_size=(64, 64)):
    """
    Carga imágenes en ESCALA DE GRISES, las redimensiona y aplana.
    """
    X = [] 
    y = [] 
    clases = ['not fractured', 'fractured'] 
    
    print(f"\nCargando {folder_path}")
    
    for label_id, nombre_clase in enumerate(clases):
        class_path = os.path.join(folder_path, nombre_clase)
        
        if not os.path.isdir(class_path):
            continue
            
        files = os.listdir(class_path)
        
        for img_name in files:
            img_path = os.path.join(class_path, img_name)
            
            if img_name.startswith('.'):
                continue

            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) 
                
                if img is not None:
                    # Redimensionar
                    img = cv2.resize(img, img_size)
                    
                    # Flatten
                    X.append(img.flatten())
                    y.append(label_id)
            except Exception as e:
                print(f"Error cargando {img_name}: {e}")

    return np.array(X), np.array(y)

In [ ]:
train_path = 'data/Bone_Fracture_Binary_Classification/train'      
val_path   = 'data/Bone_Fracture_Binary_Classification/val'
test_path  = 'data/Bone_Fracture_Binary_Classification/test'

X_train, y_train = cargar_y_aplanar(train_path)
X_val, y_val     = cargar_y_aplanar(val_path)
X_test, y_test   = cargar_y_aplanar(test_path)

print(f"\nDimensiones Train: {X_train.shape}")

scaler = StandardScaler()

# Ajustamos el scaler con train, y transformamos los demás
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

Procesando datos...

Cargando data/Bone_Fracture_Binary_Classification/train (Modo Escala de Grises)

Cargando data/Bone_Fracture_Binary_Classification/val (Modo Escala de Grises)

Cargando data/Bone_Fracture_Binary_Classification/test (Modo Escala de Grises)

Dimensiones Train: (9246, 4096)


In [23]:
from sklearn.decomposition import PCA
print(f"\nDimensiones antes de PCA: {X_train_scaled.shape[1]}") 

# componentes necesarios para conservar el 95% de la varianza
pca = PCA(n_components=0.95, random_state=42)

# Ajustamos PCA solo con el set de entrenamiento
X_train_pca = pca.fit_transform(X_train_scaled)

# Transformamos los otros sets con el MISMO pca
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"Dimensiones después de PCA: {X_train_pca.shape[1]}") 
print("\n Training ")
model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

start_train = time.time()
model.fit(X_train_pca, y_train) 
end_train = time.time()

print(f"Tiempo de entrenamiento: {end_train - start_train:.2f} segundos")



Dimensiones antes de PCA: 4096
Dimensiones después de PCA: 307

 Training 
Tiempo de entrenamiento: 8.84 segundos


In [24]:
y_pred_train = model.predict(X_train_pca)
y_pred_val   = model.predict(X_val_pca)
y_pred_test  = model.predict(X_test_pca)

acc_train = accuracy_score(y_train, y_pred_train)
acc_val   = accuracy_score(y_val, y_pred_val)
acc_test  = accuracy_score(y_test, y_pred_test)

print(f"Accuracy Train:      {acc_train*100:.2f}%")
print(f"Accuracy Validation: {acc_val*100:.2f}%")
print(f"Accuracy Test:       {acc_test*100:.2f}%")


Accuracy Train:      99.19%
Accuracy Validation: 97.35%
Accuracy Test:       97.23%


In [ ]:

print("\nMidiendo tiempo de inferencia (Promedio de 100 predicciones)...")
sample_pca = X_test_pca[0].reshape(1, -1) 

times = []
for _ in range(100):
    t_start = time.time()
    
    model.predict(sample_pca) 
    
    t_end = time.time()
    times.append(t_end - t_start)

avg_time = np.mean(times)
print(f"Inferencia promedio: {avg_time:.6f} segundos")
print(f"Desviación estándar: {np.std(times):.6f} segundos")


n_support_vectors = model.support_vectors_.shape[0]
n_features_pca = X_test_pca.shape[1]

# Fórmula para SVM con kernel RBF
flops = 2 * n_support_vectors * n_features_pca
print(f"\n Métricas Técnicas ")
print(f"Dimensiones de entrada (PCA): {n_features_pca}")
print(f"Vectores de Soporte: {n_support_vectors}")
print(f"FLOPS estimados por predicción: {flops:,}")


Midiendo tiempo de inferencia (Promedio de 100 predicciones)...
Inferencia promedio: 0.001316 segundos
Desviación estándar: 0.000868 segundos

 Métricas Técnicas 
Dimensiones de entrada (PCA): 307
Vectores de Soporte: 2818
FLOPS estimados por predicción: 1,730,252
